In [1]:
#Lorem ipsum
!pip install lorem-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# HKDF
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

In [3]:
import time

In [4]:
from lorem_text import lorem

def generar_lorem(num_palabras):
    texto = lorem.words(num_palabras)
    return texto

##1. Leer un archivo con el texto del mensaje a cifrar.

In [6]:
inicio = time.time()

mensaje = generar_lorem(10).encode()

fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion*1000, "milisegundos")

Tiempo de ejecución: 0.16307830810546875 milisegundos


##2. Generar e imprimir la(las) claves de cifrado y/o descifrado.

In [7]:
inicio = time.time()

private_key1 = ec.generate_private_key(ec.SECP256R1(), default_backend())
public_key1 = private_key1.public_key()

private_key2 = ec.generate_private_key(ec.SECP256R1(), default_backend())
public_key2 = private_key2.public_key()

shared_key1 = private_key1.exchange(ec.ECDH(), public_key2)
shared_key2 = private_key2.exchange(ec.ECDH(), public_key1)

print(private_key1)
print(public_key1)
print(private_key2)
print(public_key2)
print(shared_key1)
print(shared_key2)

fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion*1000, "milisegundos")

b'\xb7\xc1\xb1\xe0%\xd8\xa38\xa7\x9e\xd1\xce\xef\xcc\x84O\xa1=\xfd\xb1\xbbY7\x0b\xa49\x92\xf01IP$'
b'\xb7\xc1\xb1\xe0%\xd8\xa38\xa7\x9e\xd1\xce\xef\xcc\x84O\xa1=\xfd\xb1\xbbY7\x0b\xa49\x92\xf01IP$'
Tiempo de ejecución: 4.496097564697266 milisegundos


##3. Cifrar e imprimir el texto.

In [14]:
inicio = time.time()

hkdf = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'session_key',
    backend=default_backend()
)
session_key = hkdf.derive(shared_key1)

nonce = os.urandom(16)
cipher = Cipher(algorithms.AES(session_key), modes.CTR(nonce), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(mensaje) + encryptor.finalize()

print(ciphertext.hex())

fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion*1000, "milisegundos")

5205e2c8a014714654a819d92821de6f794c7a881276afff30391e93ffa2d96f23f0a49ae52a6ab928e50fb3c41806ee40c2dd7096c73293e215f762b7b09fb6584e93cb88a2e2a1c093c0a14497f8d7
Tiempo de ejecución: 2.072572708129883 milisegundos


##4. Descifrar e imprimir el texto.

In [16]:
inicio = time.time()

decryptor = cipher.decryptor()
decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()

if decrypted_text == mensaje:
  print("Descifrado correcto")

print(decrypted_text.decode())

fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion*1000, "milisegundos")

Descifrado correcto
cupiditate ratione enim tenetur temporibus accusamus ex doloremque dolores nulla
Tiempo de ejecución: 2.264738082885742 milisegundos
